# Modélisation du réseau

### Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json 
import seaborn as sns
import utm

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression

from utils import *

import os

### Parameters

In [2]:
tension_level = ['400kV', '225kV','90kV', '63kV', '45kV', '<45kV']
tension_layer = 0

## Datas 

### Lignes

1. Aériennes

In [3]:
path_ligne_aerienne = '/Users/matthiasmolenat/repos/congestion/data/2412_lignes-aeriennes-rte-nv.csv'
df_ligne_RT_a= pd.read_csv(path_ligne_aerienne,delimiter=';')

2. Souterraines

In [4]:
path_ligne_souterraine = '/Users/matthiasmolenat/repos/congestion/data/2412_lignes-souterraines-rte-nv.csv'
df_ligne_RT_s= pd.read_csv(path_ligne_souterraine,delimiter=';')

3. Concatennation

In [5]:
df_ligne_RT = pd.concat([df_ligne_RT_a,df_ligne_RT_s])

In [6]:
# calcul de la colonne des lignes
if False :

    geo_shape_index = df_ligne_RT.columns.to_list().index("Geo Shape")

    for index, row in df_ligne_RT.iterrows():
        geo_data = json.loads(row['Geo Shape'])
        longitudes, latitudes = zip(*geo_data["coordinates"])
        df_ligne_RT.at[index, "Length (m)"] = length_from_coordinate(longitudes, latitudes)

In [7]:
df_ligne_RT.shape

(17986, 24)

In [8]:
# # Simplification : suppression des lignes en doublons
# df_ligne_RT.drop_duplicates(subset=['Code ligne 1'], keep='first', inplace=True)
# df_ligne_RT.reset_index(inplace=True)
# # Function to calculate character difference
# def char_difference(str1, str2):
#     return sum(1 for a, b in zip(str1, str2) if a != b) + abs(len(str1) - len(str2))

# # Filter rows based on the condition
# filtered_rows = []
# for i in range(len(df_ligne_RT) - 1):  # Loop until the second-to-last row
#     if char_difference(df_ligne_RT.loc[i, 'Code ligne 1'], df_ligne_RT.loc[i + 1, 'Code ligne 1']) <= 2:
#         filtered_rows.append(i)

# # Add the last row if needed
# filtered_rows.append(len(df_ligne_RT) - 1)

# # Create the new DataFrame with filtered rows
# df_ligne_RT = df_ligne_RT.loc[filtered_rows].reset_index(drop=True)



## Postes

In [9]:
path_postes= '/Users/matthiasmolenat/repos/congestion/data/2412_postes-electriques-rte.csv'
df_postes_RT= pd.read_csv(path_postes,delimiter=';')

## Specific extraction 

Based on selected layer of tension

In [10]:
selected_tension = tension_level[:tension_layer+1]
df_postes_RT = df_postes_RT[df_postes_RT['Tension (kV)'].isin(selected_tension)]
df_ligne_RT = df_ligne_RT[df_ligne_RT['TENSION'].isin(selected_tension)]
df_ligne_RT.reset_index(inplace=True)


## Exploitation

Network as a graph

Surpiquage missing

In [12]:
from scipy.spatial import distance_matrix
from scipy.sparse import csr_matrix

# Extract the coordinates of the extremities
coordinates = []
for index, row in df_ligne_RT.iterrows():
    geo_data = json.loads(row['Geo Shape'])
    coordinates.append(geo_data["coordinates"][0])  # Start point
    coordinates.append(geo_data["coordinates"][-1])  # End point

# Calculate the distance matrix
dist_matrix = distance_matrix_geodesic(coordinates)

# Create a sparse matrix with edges where distance is less than or equal to 200 meters
threshold = 20  # distance in coordinates
adjacency_matrix = (dist_matrix <= threshold).astype(int)

# Convert to sparse matrix
sparse_matrix = csr_matrix(adjacency_matrix)
sparse_matrix.setdiag(0)
print(sparse_matrix)



<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12 stored elements and shape (10, 10)>
  Coords	Values
  (0, 0)	0
  (1, 1)	0
  (2, 2)	0
  (2, 4)	1
  (3, 3)	0
  (4, 2)	1
  (4, 4)	0
  (5, 5)	0
  (6, 6)	0
  (7, 7)	0
  (8, 8)	0
  (9, 9)	0


## Visualisation

In [ ]:
show_map_lines(df_ligne_RT, True)
